# Leçon 4 : L'attention

## L'ingrédient secret des GPT

Imagine que tu lis la phrase : "Le **chat** noir dort sur le **canapé**."

Si on te demande "Où dort le chat ?", ton cerveau fait automatiquement
le lien entre "dort", "chat" et "canapé" -- même si ces mots ne sont
pas côte à côte.

C'est exactement ce que fait le **mécanisme d'attention** :
il permet au modèle de regarder **n'importe quel** élément du passé,
pas seulement les derniers.

## Comment ça marche ?

Pour chaque lettre, le modèle se pose 3 questions :

1. **Query (Q)** : "Qu'est-ce que je cherche ?" (ce que cette lettre a besoin de savoir)
2. **Key (K)** : "Qu'est-ce que j'offre ?" (ce que cette lettre peut apporter)
3. **Value (V)** : "Quelle info je transmets ?" (l'information réelle)

L'attention = comparer chaque Query avec toutes les Keys pour trouver
les lettres les plus utiles, puis collecter leurs Values.

In [ ]:
import math
import random

random.seed(42)

# Simulons un mot simple
mot = "chat"
print(f"Mot : '{mot}'")
print(f"Positions : {list(enumerate(mot))}")
print()

# Chaque lettre a un embedding (simplifié à 4 dimensions)
DIM = 4
emb = {
    'c': [1.0, 0.0, 0.5, -0.3],
    'h': [0.2, 0.8, -0.1, 0.5],
    'a': [0.5, 0.3, 0.9, 0.1],
    't': [-0.3, 0.6, 0.2, 0.8],
}

for c, v in emb.items():
    print(f"  '{c}' -> {v}")

In [ ]:
# Pour simplifier, on utilise les embeddings directement comme Q, K, V
# (en vrai, il y a des matrices de transformation)

def produit_scalaire(a, b):
    """Mesure la similarité entre deux vecteurs."""
    return sum(x * y for x, y in zip(a, b))

def softmax(scores):
    """Transforme les scores en probabilités."""
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]

# Calculons l'attention pour la lettre 't' (dernière position)
# Question : quelles lettres précédentes sont importantes pour prédire
# ce qui vient après 't' dans 'chat' ?

query = emb['t']  # Ce que 't' cherche

# Comparer avec toutes les lettres précédentes (y compris elle-même)
scores = []
for c in mot:
    key = emb[c]  # Ce que chaque lettre offre
    score = produit_scalaire(query, key) / math.sqrt(DIM)
    scores.append(score)

print("Scores d'attention pour 't' :")
for c, s in zip(mot, scores):
    print(f"  '{c}' : {s:.2f}")

# Transformer en probabilités
poids_attention = softmax(scores)
print()
print("Poids d'attention (après softmax) :")
for c, w in zip(mot, poids_attention):
    barre = '#' * int(w * 30)
    print(f"  '{c}' : {w:.1%} {barre}")

print()
print("Le modèle 'regarde' plus les lettres avec un poids élevé !")

In [ ]:
# Collecter l'information (somme pondérée des Values)

resultat = [0.0] * DIM
for c, w in zip(mot, poids_attention):
    value = emb[c]
    for d in range(DIM):
        resultat[d] += w * value[d]

print("Vecteur de sortie de l'attention :")
print(f"  {[f'{x:.2f}' for x in resultat]}")
print()
print("Ce vecteur combine l'information de toutes les lettres,")
print("en donnant plus de poids aux lettres les plus pertinentes.")
print()
print("C'est cette information qui sera utilisée pour prédire")
print("la lettre suivante !")

## Attention causale : pas de triche !

Règle importante : quand le modèle prédit la lettre suivante,
il **ne peut pas regarder le futur** -- seulement le passé.

```
Pour prédire après 'c' : peut regarder [c]
Pour prédire après 'h' : peut regarder [c, h]
Pour prédire après 'a' : peut regarder [c, h, a]
Pour prédire après 't' : peut regarder [c, h, a, t]
```

On appelle ça le **masque causal**. C'est ce qui fait de GPT un modèle
**auto-régressif** : il génère un token à la fois, de gauche à droite.

In [ ]:
# Visualisons le masque causal
print("Masque causal pour 'chat' :")
print()
print("          c    h    a    t")
for i, c in enumerate(mot):
    row = ""
    for j in range(len(mot)):
        if j <= i:
            row += "  OK "
        else:
            row += "  -- "
    print(f"  {c} : {row}")

print()
print("OK = peut regarder, -- = interdit (c'est le futur)")

## Multi-têtes : regarder de plusieurs façons

En pratique, on utilise **plusieurs têtes d'attention** en parallèle.
Chaque tête apprend à chercher un type d'information différent :

- Tête 1 : quelles lettres forment des syllabes ensemble ?
- Tête 2 : quelle est la voyelle la plus récente ?
- Tête 3 : est-ce que c'est un début ou une fin de mot ?

Les résultats de toutes les têtes sont combinés pour une prédiction finale.

Dans microgpt.py de Karpathy, il y a **4 têtes** d'attention.

## Résumé visuel

```
Lettres d'entrée :  [c] [h] [a] [t]
                     |   |   |   |
                     v   v   v   v
Embeddings :        [---] [---] [---] [---]
                     |   |   |   |
                     v   v   v   v
Attention :         Chaque lettre regarde les précédentes
                    et collecte l'info importante
                     |   |   |   |
                     v   v   v   v
Prédiction :        Quelle est la prochaine lettre ?
```

## Ce qu'on a appris

- L'**attention** permet au modèle de regarder toutes les lettres précédentes, pas juste la dernière
- Ça fonctionne avec **Q** (je cherche), **K** (j'offre), **V** (mon info)
- Le **masque causal** empêche de tricher en regardant le futur
- Plusieurs **têtes** d'attention regardent des choses différentes en parallèle

### Dernière étape

On a maintenant toutes les pièces du puzzle. Dans la prochaine leçon,
on assemble tout pour construire un vrai mini-LLM !

---
*Prochaine leçon : [05 - Mon premier LLM](05_mon_premier_llm.ipynb)*

---

### Sources (ISO 42001)

- **Mécanisme Q/K/V et masque causal** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy, section self-attention
- **"Attention Is All You Need"** : Vaswani et al., 2017, [arXiv:1706.03762](https://arxiv.org/abs/1706.03762) — article fondateur des Transformers
- **Explication visuelle de l'attention** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023), section attention
- **Multi-head attention (4 têtes dans microgpt.py)** : même source, paramètre `n_head=4`